In [72]:
import psycopg2
import pandas as pd
pd.options.display.max_columns = 999

In [ ]:
user = ''
password = ''
ip = ''
port = ''
dbname = ''

In [3]:
db_connection_string = 'postgresql://{}:{}@{}:{}/{}'

with psycopg2.connect(db_connection_string.format(user,
                                                  password,
                                                  ip,
                                                  port,
                                                  dbname)) as con:
    cursor = con.cursor()

    sql = """SELECT * from chess_games;"""
    cursor.execute(sql)
    colnames = [desc.name for desc in cursor.description]
    results = cursor.fetchall()

In [23]:
df = pd.DataFrame.from_records(results, columns=colnames)

In [22]:
df.head()

,id,black,black_elo,black_rating_diff,date_played,opening_played,event_type,result,round,game_link,termination,time_control,utc_date_played,time_played,chess_variant,white,white_elo,white_rating_diff
0,1,arsenic14,1173.0,NaN,2019-07-07,D00,Casual Blitz game,1-0,-,https://lichess.org/UN3ZiugD,Normal,300+3,2019-07-07,16:18:07,Standard,Gilbuu,1567.0,NaN
1,2,Gilbuu,1567.0,NaN,2019-07-07,A40,Casual Blitz game,0-1,-,https://lichess.org/lmulopuY,Normal,300+3,2019-07-07,16:12:14,Standard,arsenic14,1173.0,NaN
2,3,arsenic14,1173.0,NaN,2019-07-05,C10,Casual Blitz game,1-0,-,https://lichess.org/8MyY1oB9,Normal,300+3,2019-07-05,19:24:09,Standard,Gilbuu,1598.0,NaN
3,4,arsenic14,1173.0,NaN,2019-07-02,B15,Casual Blitz game,1-0,-,https://lichess.org/r9BwkTLr,Normal,300+3,2019-07-02,19:27:42,Standard,Gilbuu,1593.0,NaN
4,5,arsenic14,1173.0,NaN,2019-06-26,B07,Casual Blitz game,1-0,-,https://lichess.org/41gYrIXp,Normal,300+0,2019-06-26,03:30:30,Standard,Gilbuu,1432.0,NaN


In [47]:
import seaborn as sns
from copy import deepcopy

In [105]:
player = 'siddhartha13'

In [106]:
tbo_games = deepcopy(df[(df['black'] == player) | (df['white'] == player)])

In [107]:
srs = tbo_games['black'] == player
tbo_games['tbo_color'] = srs.map({True: 'black',
                                  False: 'white'})

In [108]:
tbo_games['tbo_rating'] = (tbo_games['black_elo'] * srs) + (tbo_games['white_elo'] * ~srs)

In [111]:
tbo_games['datetime_played'] = pd.to_datetime(tbo_games['date_played'].astype(str) + ' ' + tbo_games['time_played'].astype(str))

In [112]:
tbo_games['datetime_played_s'] = tbo_games['datetime_played'].astype('int64') / 1000000

In [113]:
tbo_games['game_type'] = tbo_games['event_type'].str.extract('.+? (.+?) .+?')

In [114]:
tbo_games.head()

,id,black,black_elo,black_rating_diff,date_played,opening_played,event_type,result,round,game_link,termination,time_control,utc_date_played,time_played,chess_variant,white,white_elo,white_rating_diff,tbo_color,tbo_rating,datetime_played,datetime_played_s,game_type
491,492,siddhartha13,1790.0,NaN,2019-06-01,B20,Casual Blitz game,0-1,-,https://lichess.org/IQu4dIRt,Normal,180+1,2019-06-01,13:06:17,Standard,Grahtbo,1301.0,NaN,black,1790.0,2019-06-01 13:06:17,1.559394e+12,Blitz
636,637,siddhartha13,1803.0,NaN,2019-05-21,A45,Casual Blitz game,1/2-1/2,-,https://lichess.org/Nk00BfRq,Normal,180+1,2019-05-21,14:16:09,Standard,Grahtbo,1374.0,NaN,black,1803.0,2019-05-21 14:16:09,1.558448e+12,Blitz
738,739,siddhartha13,1669.0,NaN,2019-05-06,A46,Casual Blitz game,1/2-1/2,-,https://lichess.org/2IoZ6ECI,Normal,180+0,2019-05-06,15:30:48,Standard,Grahtbo,1364.0,NaN,black,1669.0,2019-05-06 15:30:48,1.557157e+12,Blitz
1186,1187,siddhartha13,1677.0,-9.0,2019-03-19,A45,Rated Blitz game,1/2-1/2,-,https://lichess.org/ySDJEOuX,Normal,180+0,2019-03-19,12:18:47,Standard,Grahtbo,1255.0,9.0,black,1677.0,2019-03-19 12:18:47,1.552998e+12,Blitz
1187,1188,Grahtbo,1235.0,20.0,2019-03-19,D00,Rated Blitz game,0-1,-,https://lichess.org/9NhdYKKd,Normal,180+0,2019-03-19,12:13:48,Standard,siddhartha13,1697.0,-20.0,white,1697.0,2019-03-19 12:13:48,1.552998e+12,Blitz


In [1]:
filter_srs = pd.Timestamp.today() - tbo_games['datetime_played'] < pd.Timedelta('180 days')

g = sns.lmplot(x='datetime_played_s',
                 y='tbo_rating',
                 hue='game_type',
                # legend=False,
                 data=tbo_games[filter_srs])
xticks = g.get_xticks()
g.set_xticklabels([pd.to_datetime(tm, unit='ms').strftime('%Y-%m-%d\n %H:%M:%S') for tm in xticks], rotation=50)

NameError: name 'pd' is not defined